# Evaluate model endpoints using Azure AI Evaluation APIs

## Objective

This tutorial provides a step-by-step guide on how to evaluate prompts against variety of model endpoints deployed on Azure AI Platform or non Azure AI platforms. 

This guide uses Python Class as an application target which is passed to Evaluate API provided by PromptFlow SDK to evaluate results generated by LLM models against provided prompts. 

This tutorial uses the following Azure AI services:

- [azure-ai-evaluation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk)

## Time

You should expect to spend 30 minutes running this sample. 

## About this example

This example demonstrates evaluating model endpoints responses against provided prompts using azure-ai-evaluation

## Before you begin

### Installation

Install the following packages required to execute this notebook. 

In [2]:
%pip install azure-ai-evaluation
%pip install promptflow-azure
%pip install promptflow-tracing

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Parameters and imports

In [1]:
from pprint import pprint

import pandas as pd
import random

## Target Application

We will use Evaluate API provided by Prompt Flow SDK. It requires a target Application or python Function, which handles a call to LLMs and retrieve responses. 

In the notebook, we will use an Application Target `ModelEndpoints` to get answers from multiple model endpoints against provided question aka prompts. 

This application target requires list of model endpoints and their authentication keys. For simplicity, we have provided them in the `env_var` variable which is passed into init() function of `ModelEndpoints`.

In [ ]:
env_var = {
    "gpt4-0613": {
        "endpoint": "https://ai-***.**.azure.com/openai/deployments/gpt-4/chat/completions?api-version=2023-03-15-preview",
        "key": "***",
    },
    "gpt35-turbo": {
        "endpoint": "https://ai-**.openai.azure.com/openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-03-15-preview",
        "key": "***",
    },
    "mistral7b": {
        "endpoint": "https://***.eastus.inference.ml.azure.com/v1/chat/completions",
        "key": "***",
    },
    "phi3_mini_serverless": {
        "endpoint": "https://Phi-3-mini-4k-instruct-rpzhe.eastus2.models.ai.azure.com/v1/chat/completions",
        "key": "***",
    },
    "tiny_llama": {
        "endpoint": "https://api-inference.huggingface.co/models/TinyLlama/TinyLlama-1.1B-Chat-v1.0/v1/chat/completions",
        "key": "***",
    },
    "gpt2": {
        "endpoint": "https://api-inference.huggingface.co/models/openai-community/gpt2",
        "key": "***",
    },
}


Please provide Azure AI Project details so that traces and eval results are pushing in the project in Azure AI Studio.

In [ ]:
azure_ai_project = {
    "subscription_id": "<your-subscription-id>",
    "resource_group_name": "<your-resource-group-name>",
    "project_name": "<your-project-name>",
}

In [ ]:
import os

# Use the following code to set the environment variables if not already set. If set, you can skip this step.

os.environ["AZURE_OPENAI_API_KEY"] = "<your-api-key>"
os.environ["AZURE_OPENAI_API_VERSION"] = "<api version>"
os.environ["AZURE_OPENAI_DEPLOYMENT"] = "<your-deployment>"
os.environ["AZURE_OPENAI_ENDPOINT"] = "<your-endpoint>"

## Model Endpoints
The following code demonstrates how to call various model endpoints, and is configured based on `env_var` set above. For any model in `env_var`, if you do not have that model deployed in your AI project, please comment it out. If you have a model that you would like to test that does not correspond with one of the types seen below, please include that type in the `__call__` function and create a helper function to call the model's endpoint via REST. 

In [3]:
import requests
from typing_extensions import Self
from typing import TypedDict
from promptflow.tracing import trace


class ModelEndpoints:
    def __init__(self: Self, env: dict, model_type: str) -> str:
        self.env = env
        self.model_type = model_type

    class Response(TypedDict):
        query: str
        response: str

    @trace
    def __call__(self: Self, query: str) -> Response:
        if self.model_type == "gpt4-0613":
            output = self.call_gpt4_endpoint(query)
        elif self.model_type == "gpt35-turbo":
            output = self.call_gpt35_turbo_endpoint(query)
        elif self.model_type == "mistral7b":
            output = self.call_mistral_endpoint(query)
        elif self.model_type == "tiny_llama":
            output = self.call_tiny_llama_endpoint(query)
        elif self.model_type == "phi3_mini_serverless":
            output = self.call_phi3_mini_serverless_endpoint(query)
        elif self.model_type == "gpt2":
            output = self.call_gpt2_endpoint(query)
        else:
            output = self.call_default_endpoint(query)

        return output

    def query(self: Self, endpoint: str, headers: str, payload: str) -> str:
        response = requests.post(url=endpoint, headers=headers, json=payload)
        return response.json()

    def call_gpt4_endpoint(self: Self, query: str) -> Response:
        endpoint = self.env["gpt4-0613"]["endpoint"]
        key = self.env["gpt4-0613"]["key"]

        headers = {"Content-Type": "application/json", "api-key": key}

        payload = {"messages": [{"role": "user", "content": query}], "max_tokens": 500}

        output = self.query(endpoint=endpoint, headers=headers, payload=payload)
        response = output["choices"][0]["message"]["content"]
        return {"query": query, "response": response}

    def call_gpt35_turbo_endpoint(self: Self, query: str) -> Response:
        endpoint = self.env["gpt35-turbo"]["endpoint"]
        key = self.env["gpt35-turbo"]["key"]

        headers = {"Content-Type": "application/json", "api-key": key}

        payload = {"messages": [{"role": "user", "content": query}], "max_tokens": 500}

        output = self.query(endpoint=endpoint, headers=headers, payload=payload)
        response = output["choices"][0]["message"]["content"]
        return {"query": query, "response": response}

    def call_tiny_llama_endpoint(self: Self, query: str) -> Response:
        endpoint = self.env["tiny_llama"]["endpoint"]
        key = self.env["tiny_llama"]["key"]

        headers = {"Content-Type": "application/json", "Authorization": ("Bearer " + key)}

        payload = {
            "model": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
            "messages": [{"role": "user", "content": query}],
            "max_tokens": 500,
            "stream": False,
        }

        output = self.query(endpoint=endpoint, headers=headers, payload=payload)
        response = output["choices"][0]["message"]["content"]
        return {"query": query, "response": response}

    def call_phi3_mini_serverless_endpoint(self: Self, query: str) -> Response:
        endpoint = self.env["phi3_mini_serverless"]["endpoint"]
        key = self.env["phi3_mini_serverless"]["key"]

        headers = {"Content-Type": "application/json", "Authorization": ("Bearer " + key)}

        payload = {"messages": [{"role": "user", "content": query}], "max_tokens": 500}

        output = self.query(endpoint=endpoint, headers=headers, payload=payload)
        response = output["choices"][0]["message"]["content"]
        return {"query": query, "response": response}

    def call_gpt2_endpoint(self: Self, query: str) -> Response:
        endpoint = self.env["gpt2"]["endpoint"]
        key = self.env["gpt2"]["key"]

        headers = {"Content-Type": "application/json", "Authorization": ("Bearer " + key)}

        payload = {
            "inputs": query,
        }

        output = self.query(endpoint=endpoint, headers=headers, payload=payload)
        response = output[0]["generated_text"]
        return {"query": query, "response": response}

    def call_mistral_endpoint(self: Self, query: str) -> Response:
        endpoint = self.env["mistral7b"]["endpoint"]
        key = self.env["mistral7b"]["key"]

        headers = {"Content-Type": "application/json", "Authorization": ("Bearer " + key)}

        payload = {"messages": [{"content": query, "role": "user"}], "max_tokens": 50}

        output = self.query(endpoint=endpoint, headers=headers, payload=payload)
        response = output["choices"][0]["message"]["content"]
        return {"query": query, "response": response}

    def call_default_endpoint(query: str) -> Response:
        return {"query": "What is the capital of France?", "response": "Paris"}

## Data

Following code reads Json file "data.jsonl" which contains inputs to the Application Target function. It provides question, context and ground truth on each line. 

In [ ]:
df = pd.read_json("data.jsonl", lines=True)
print(df.head())

## Configuration
To use Relevance and Cohenrence Evaluator, we will Azure Open AI model details as a Judge that can be passed as model config.

In [ ]:
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_KEY"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
}

## Run the evaluation

The Following code runs Evaluate API and uses Content Safety, Relevance and Coherence Evaluator to evaluate results from different models.

The following are the few parameters required by Evaluate API. 

+   Data file (Prompts): It represents data file 'data.jsonl' in JSON format. Each line contains question, context and ground truth for evaluators.     

+   Application Target: It is name of python class which can route the calls to specific model endpoints using model name in conditional logic.  

+   Model Name: It is an identifier of model so that custom code in the App Target class can identify the model type and call respective LLM model using endpoint URL and auth key.  

+   Evaluators: List of evaluators is provided, to evaluate given prompts (questions) as input and output (answers) from LLM models. 

In [ ]:
import pathlib

from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    RelevanceEvaluator,
)

relevance_evaluator = RelevanceEvaluator(model_config)

models = [
    "gpt4-0613",
    "gpt35-turbo",
    "mistral7b",
    "phi3_mini_serverless",
    "tiny_llama",
    "gpt2",
]

path = str(pathlib.Path(pathlib.Path.cwd())) + "/data.jsonl"

for model in models:
    randomNum = random.randint(1111, 9999)
    results = evaluate(
        evaluation_name="Eval-Run-" + str(randomNum) + "-" + model.title(),
        data=path,
        target=ModelEndpoints(env_var, model),
        evaluators={
            "relevance": relevance_evaluator,
        },
        evaluator_config={
            "relevance": {"response": "${target.response}", "context": "${data.context}", "query": "${data.query}"},
        },
    )

View the results

In [ ]:
pprint(results)

In [ ]:
pd.DataFrame(results["rows"])